In [1]:
# Project Description: Extract the top 1000 movies list from multiple pages from IMDB
# created on 2023-08-06
# created by VM


In [2]:
# import libraries 

from bs4 import BeautifulSoup
import requests
import time
from time import sleep
import datetime
from random import randint
import pandas as pd
import numpy as np

In [3]:
#Declaring the headers 

headers = {"Accept-Language": "en-US,en;q=0.5"}

#Declare empty variables

movie_name = []
year = []
time=[]
rating=[]
metascore =[]
votes = []
gross = []
description = []


In [4]:
# Data will be extracted by concatenating the page start using variable and generating dynamic number

# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=1&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=101&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=2011&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=301&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=401&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=501&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=601&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=701&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=801&ref_=adv_nxt
# https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=901&ref_=adv_nxt
    

In [5]:
pages = np.arange(1,1000,100)


In [ ]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        #MOVIE NAMES
        movie_name.append(name)
        
        #RELEASE YEAR
        year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
        year.append(year_of_release)
        
        #TOTAL TIME
        runtime = store.p.find("span", class_ = 'runtime').text
        time.append(runtime)
        
        #RATINGS
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        #METASCORES
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "0"
        metascore.append(meta)
        
        
        #VOTES AND GROSS AMOUNTS
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else '0'
        gross.append(grosses)
        
        
        #MOVIES DESCRIPTION
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else 'NULL'
        description.append(description_)
        
        

In [ ]:
#Create a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, "Votes" : votes, "Gross": gross, "Description": description  })

In [ ]:
movie_list.head()

In [ ]:
movie_list.count()

In [ ]:
movie_list.nunique()

In [ ]:
movie_list.info()

In [ ]:
#save the data in csv format
movie_list.to_csv("IMDB_Top_1000_Movies_CSV_REPORT.csv",index=False)

In [ ]:
# Saving the data into parquet format with snappy compression
movie_list.to_parquet("IMDB_Top_1000_Movies_PARQUET_REPORT.parquet",compression='snappy', index=None)